In [1]:
import numpy as np
import pymorphy2
import string
import re
import os
import pandas as pd
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

In [2]:
# import nltk
# nltk.download('stopwords')
# C:\Users\admin\AppData\Roaming\nltk_data

In [3]:
# ds = pd.read_excel('d:/Projects/MegaJack/data/Таблица обращений итоговая_Kostya.xlsx')
# ds = ds.dropna()
# ds['ИД'] = ds['ИД'].astype('int16')

ds = pd.read_excel(r'd:\Projects\MegaJack\rosatom\data\Вид поручения\Вид поручения.xlsx')
ds = ds.dropna()

In [4]:
ds.shape[0]

30

In [5]:
ds.head()

,Вид поручения,id,Текст
0,Организационное,1,Провести инструктаж на участке
1,Организационное,1,Соблюдать требования безопасности
2,Организационное,1,Ожидание комиссии
3,Организационное,1,Носить рабочую форму одежды
4,Организационное,1,Содержать в чистоте место работы


In [6]:
charsforexcluding = string.punctuation + u'«»№•–’‘”“\n\t¬…—'
global_morph = pymorphy2.MorphAnalyzer()
stop_words = [x for x in stopwords.words('english') + stopwords.words('russian') if not x in ["не"]]
stop_words += ["коллега","просить","просьба","здравствовать","спасибо","пожалуйста","уважаемый", "уважение"]
stop_words = set(stop_words)

def get_unigramms(decision, internal_morph=None):

    if isinstance(decision, str):
        morph = global_morph if internal_morph is None else internal_morph
        text = " " + decision.lower() + " "
        text = text.replace("добрый день", "").replace("доброе утро", "")

        unigramms = ""
        # заменим пунктуацию на пробел
        for x in charsforexcluding:
            text = text.replace(x, ' ')

        for el in text.split():

            el = el.lower()
            el = el.replace(" ", "")

            if (not el.isdigit()) & (not el in stop_words):
                if el != '':

                    prs = morph.parse(el)[0]
                    nf = prs.normal_form

                    if nf not in stop_words:
                        if unigramms == " ":
                            unigramms = nf
                        else:
                            unigramms += " "
                            unigramms += nf

        return unigramms

    else:
        return ""

In [7]:
def create_unigramms_in_ds(data_source, internal_morph=None):
    
    data_source['unigramms'] = data_source['Текст'].apply(lambda text: get_unigramms(text, internal_morph))
    data_source = data_source[~data_source['unigramms'].isnull()]

    return data_source

In [8]:
ds = create_unigramms_in_ds(ds)

In [9]:
ds

,Вид поручения,id,Текст,unigramms
0,Организационное,1,Провести инструктаж на участке,провести инструктаж участок
1,Организационное,1,Соблюдать требования безопасности,соблюдать требование безопасность
2,Организационное,1,Ожидание комиссии,ожидание комиссия
3,Организационное,1,Носить рабочую форму одежды,носить рабочий форма одежда
4,Организационное,1,Содержать в чистоте место работы,содержимый чистота место работа
5,Организационное,1,Следить за исполнением требований техники безо...,следить исполнение требование техника безопас...
6,Организационное,1,Отчитаться перед руководством,отчитаться руководство
7,Организационное,1,Подготовить территорию,подготовить территория
8,Организационное,1,Встретить комиссию,встретить комиссия
9,Организационное,1,Следить за чистотой на участке,следить чистота участок


### Делим выборку

In [10]:
CLASS_FILED = 'id'
# CLASS_FILED = 'Вид поручения'

In [11]:
'Кол-во коассов: {}'.format(len(ds[CLASS_FILED].unique()))

'Кол-во коассов: 3'

In [12]:
train, test = train_test_split(ds[CLASS_FILED], random_state=1)

In [13]:
ds_train = ds.loc[train.index]
ds_test = ds.loc[test.index]

In [14]:
count_features = 3000
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True, ngram_range=(1, 2), max_features=count_features)
features_train = vect.fit_transform(ds_train['unigramms'])
features_test = vect.transform(ds_test['unigramms'])

In [15]:
model = LogisticRegression(C=5, random_state=1, solver='liblinear', multi_class='ovr')
model.fit(features_train, ds_train[CLASS_FILED])

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=1,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [16]:
acc_train = metrics.accuracy_score(ds_train[CLASS_FILED], model.predict(features_train))
acc_test = metrics.accuracy_score(ds_test[CLASS_FILED], model.predict(features_test))

print('acc_train', acc_train)
print('acc_test', acc_test)

acc_train 1.0
acc_test 0.5


In [17]:
# чистый пердикт
# ds_test['pred'] = model.predict(features_test)

In [18]:
# предикт с вероятностями
probas = model.predict_proba(features_test)
class_indexes = np.argmax(probas, axis=1)  # индексы классов с max вероятностью
class_probas = probas[np.arange(features_test.shape[0]), class_indexes]
class_output = model.classes_[class_indexes]
ds_test[f'{CLASS_FILED} prob'] = class_probas
ds_test[f'{CLASS_FILED} pred'] = class_output
ds_test[f'{CLASS_FILED} success'] = (ds_test[f'{CLASS_FILED} pred'] == ds_test[CLASS_FILED]).astype('int16')

In [19]:
def create_conf_report(ds: pd.DataFrame):
    col_name = CLASS_FILED
    prob_col = f'{CLASS_FILED} prob'
    success_col = f'{CLASS_FILED} success'

    ds_rep = ds.sort_values(by=[prob_col], ascending=[False])
    ds_rep['{} Кол-во'.format(col_name)] = range(1, len(ds_rep) + 1)
    ds_rep['{} Соотв-сум'.format(col_name)] = ds_rep[success_col].cumsum()
    ds_rep['{} Точность'.format(col_name)] = ds_rep['{} Соотв-сум'.format(col_name)] / ds_rep['{} Кол-во'.format(col_name)]
    ds_rep['{} Доля'.format(col_name)] = ds_rep['{} Кол-во'.format(col_name)] / len(ds_rep)

    return ds_rep

In [20]:
ds_test = create_conf_report(ds_test)

In [21]:
ds_test.to_excel(r'd:\Projects\MegaJack\rosatom\data\Вид поручения\Вид поручения - Результат обучения - {}.xlsx'.format(CLASS_FILED))

### Обучаем на всех данных

In [22]:
model = LogisticRegression(C=5, random_state=1, solver='liblinear', multi_class='ovr')
features_all = vect.fit_transform(ds['unigramms'])
model.fit(features_all, ds[CLASS_FILED])

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=1,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
pickle.dump(vect, open(r'd:\Projects\MegaJack\rosatom\data\Вид поручения\vect.pickle', 'wb'))
pickle.dump(model, open(r'd:\Projects\MegaJack\rosatom\data\Вид поручения\model.pickle', 'wb'))

### Тестируем

In [24]:
text = 'предоставить отчет'
unigramms = get_unigramms(text)
unigramms_vect = vect.transform([unigramms])
result = model.predict(unigramms_vect)

In [25]:
result[0]

1

In [26]:
result = model.predict_proba(unigramms_vect)
result

array([[0.35495051, 0.30732882, 0.33772066]])